In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya

/home/husein/dev/malaya/malaya/tokenizer.py:208: FutureWarning: Possible nested set at position 3372
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:208: FutureWarning: Possible nested set at position 3890
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [7]:
from malaya.text.rules import rules_normalizer, rules_compound_normalizer
from malaya.text.normalization import _is_number_regex
from collections import defaultdict
import re
import random
import json

In [8]:
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/normalization/en-lexicon/en-social-media-lexicon.json
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/normalization/en-lexicon/en-social-media-lexicon-v2.json

In [11]:
with open('en-social-media-lexicon.json') as fopen:
    en_lexicon = json.load(fopen)
    
with open('en-social-media-lexicon-v2.json') as fopen:
    en_lexicon_v2 = json.load(fopen)

In [13]:
compound_normalizer = defaultdict(list)
normalizer = defaultdict(list)

for k, v in en_lexicon.items():
    if not len(v):
        continue
    if len(k.split()) > 1:
        compound_normalizer[k].extend(v)
    else:
        normalizer[k].extend(v)

for k, v in en_lexicon_v2.items():
    if not len(v):
        continue
    if len(k.split()) > 1:
        compound_normalizer[k].extend(v)
    else:
        normalizer[k].extend(v)
        
compound_normalizer_regex = (
    '(?:' + '|'.join(list(compound_normalizer.keys())) + ')'
)

In [14]:
# def _replace_compound(string, 
#                       rules_regex = rules_compound_normalizer_regex, 
#                       rules = rev_rules_compound_normalizer):
#     results = re.findall(rules_regex, string, flags=re.IGNORECASE
#     )
#     for r in results:
#         try:
#             string = string.replace(r, random.choice(rules[r.lower()]))
#         except:
#             pass
#     return string

def _replace_compound(string, rules = compound_normalizer):
    for k in list(rules.keys()):
        results = [(m.start(0), m.end(0)) for m in re.finditer(k, string, flags=re.IGNORECASE)]
        for r in results:
            sub = string[r[0]: r[1]]
            try:
                replaced = random.choice(rules[sub.lower()])
                if replaced:
                    if r[1] < len(string) and string[r[1]] != ' ':
                        continue
                    if r[0] - 1 > len(string) and string[r[0] - 1] != ' ':
                        continue

                    sub = case_of(sub)(replaced)
                    string = string[:r[0]] + sub + string[r[1]:]
            except:
                pass
    return string

In [15]:
_replace_compound('sudah tentu tidak')

'sudah tentu tidak'

In [17]:
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

def strip_punct(word):
    left = []
    right = []
    i = 0
    while i < len(word):
        if word[i] in PUNCTUATION:
            left.append(word[i])
            i += 1
        else:
            break
    i = len(word) - 1
    while i > 0:
        if word[i] in PUNCTUATION:
            right.append(word[i])
            i -= 1
        else:
            break
    left = ''.join(left)
    right = ''.join(right[::-1])
    if len(right):
        word_ = word[:-len(right)]
    else:
        word_ = word
    word_ = word_[len(left):]
    return left, right, word_


def replace_shortword(word, rules = normalizer):
    left, right, word_ = strip_punct(word)
    word_ = word_[len(left):]
    lower_word_ = word_.lower()
    if lower_word_ in rules:
        word_ = case_of(word_)(random.choice(rules[lower_word_]))
        word_ = f'{left}{word_}{right}'
        return word_
    else:
        return word
    
strip_punct('counters,')

('', ',', 'counters')

In [18]:
_replace_compound('i like, text to cell', rules = compound_normalizer)

'i like, ttc'

In [19]:
import json

with open('/home/husein/translation/alignment.alignment') as fopen:
    for i in fopen:
        data = json.loads(i)
        break

In [20]:
data['left'][0]

'I will fight with them till they become like us."'

In [21]:
[replace_shortword(word) for word in data['left'][0].split()]

['A',
 'wiull',
 'fiter',
 'wiy',
 'ehem',
 'til',
 'ther',
 'be_come',
 'likee',
 'hus."']

In [22]:
def replace_words_punct(left_word, right_word):
    left_left, left_right, left_word = strip_punct(left_word)
    right_left, right_right, right_word = strip_punct(right_word)
    return f'{left_left}{right_word}{left_right}'

def random_replace_alignment(left, right, alignment, min_replace = 2, max_replace = 7):
    splitted_left = left.split()
    splitted_right = right.split()
    
    selected_alignment = []
    for s in alignment:
        l = s[0]
        r = s[1]
        if _is_number_regex(splitted_left[l].replace(',', '').replace('.', '')) or _is_number_regex(splitted_right[r].replace(',', '').replace('.', '')):
            continue
        elif splitted_left[l].isupper() or splitted_right[r].isupper():
            continue
        elif splitted_left[l].istitle() or splitted_right[r].istitle():
            continue
        elif splitted_left[l] == splitted_right[r]:
            continue
        elif splitted_right[r].lower() in ['the', 'a', 'an', 'it', 'is', 'are']:
            continue
        else:
            selected_alignment.append((l, r))
    
    count_replace = random.randint(min_replace, min(max_replace, len(selected_alignment)))
    
    selected = random.sample(selected_alignment, count_replace)
    for s in selected:
        splitted_left[s[0]] = replace_words_punct(splitted_left[s[0]], splitted_right[s[1]])
    
    return ' '.join(splitted_left)

In [23]:
import copy

consonants = 'bcdfghjklmnpqrstvwxyz'

def augment(left, right, alignment, p_replace_shortword = 0.7, p_vowel_alternate = 0.8):
    
    left = _replace_compound(left, rules = copy.deepcopy(compound_normalizer))
    left = [(replace_shortword(word, rules = normalizer), False) if random.random() > p_replace_shortword else (word, True) for word in left.split()]
    left_ = []
    for l in left:
        if _is_number_regex(l[0].replace(',', '').replace('.', '')):
            left_.append(l[0])
            continue
        if l[0].isupper():
            left_.append(l[0])
            continue
        if l[0].istitle():
            left_.append(l[0])
            continue
        
        if l[1]:
            s = l[0]
                
            if random.random() > p_vowel_alternate:
                try:
                    s = malaya.augmentation.vowel_alternate(s)
                except:
                    pass
                try:
                    if random.random() and s[-1] == 'a' and s[-2] in consonants:
                        s = s[:-1] + 'e'
                except:
                    pass
        else:
            s = l[0]
        
        left_.append(case_of(l[0])(s))
    return ' '.join(left_)

In [30]:
alignment = data['forward'][1]
left = data['left'][1]
right = data['right'][1]
augment(left, right, alignment)

'19:15 (Asad) Hence, peace was apen him on the stay when he was born,'

In [32]:
from tqdm import tqdm

with open('en-ms-augment-v1.jsonl', 'w') as fwrite:
    with open('/home/husein/translation/alignment.alignment') as fopen:
        for i in tqdm(fopen):
            try:
                data = json.loads(i)
                result = []
                for k in range(len(data['right'])):
                    alignment = data['forward'][k]
                    left = data['left'][k]
                    right = data['right'][k]
                    try:
                        augmented = augment(left, right, alignment)
                        result.append({'left': left, 'right': right, 'augmented': augmented})
                    except:
                        pass
                fwrite.write(f'{json.dumps(result)}\n')
            except:
                pass

736635it [1:11:36, 171.44it/s]
